<a href="https://colab.research.google.com/github/BinarySstar/Credit-Card-Segment-Classfication/blob/master/Credit_Card_Segment_Classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Mount
train 및 test 데이터를 drive에 저장하기 위해 mount를 진행

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# EDA
## 데이터 로드 및 병합
- parquet 형식의 데이터를 로드
- 월별로 분리되어 있던 데이터를 하나의 parquet 파일로 merge

In [2]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
import os
from glob import glob

# Google Drive 경로 설정
base_path = "drive/MyDrive/고급프로그래밍/train"
save_path = "drive/MyDrive/고급프로그래밍/병합완료"
os.makedirs(save_path, exist_ok=True)

# 폴더 이름 정의
folder_name = ["회원정보", "신용정보", "승인매출정보", "청구입금정보", "잔액정보", "채널정보", "마케팅정보", "성과정보"]
folders = [f"{i}.{name}" for i, name in enumerate(folder_name, start=1)]

# 병합된 결과를 저장할 딕셔너리
merged_dfs = {}

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    parquet_files = glob(os.path.join(folder_path, "*.parquet"))

    output_name = f"{folder.split('.')[1]}.csv"
    output_path = os.path.join(save_path, output_name)

    first = True
    for file in parquet_files:
        try:
            df = pd.read_parquet(file, engine="fastparquet")

            # 파일 저장 (첫 파일: header 포함, 이후: append)
            df.to_csv(output_path, mode='w' if first else 'a',
                      header=first, index=False)
            first = False
            print(f"📄 저장 중: {file} → {output_path} ({df.shape[0]} rows)")

        except Exception as e:
            print(f"❌ {file} 처리 중 오류: {e}")

📄 저장 중: drive/MyDrive/고급프로그래밍/train/1.회원정보/201812_train_회원정보.parquet → drive/MyDrive/고급프로그래밍/병합완료/회원정보.csv (400000 rows)
📄 저장 중: drive/MyDrive/고급프로그래밍/train/1.회원정보/201811_train_회원정보.parquet → drive/MyDrive/고급프로그래밍/병합완료/회원정보.csv (400000 rows)
📄 저장 중: drive/MyDrive/고급프로그래밍/train/1.회원정보/201810_train_회원정보.parquet → drive/MyDrive/고급프로그래밍/병합완료/회원정보.csv (400000 rows)
📄 저장 중: drive/MyDrive/고급프로그래밍/train/1.회원정보/201809_train_회원정보.parquet → drive/MyDrive/고급프로그래밍/병합완료/회원정보.csv (400000 rows)
📄 저장 중: drive/MyDrive/고급프로그래밍/train/1.회원정보/201808_train_회원정보.parquet → drive/MyDrive/고급프로그래밍/병합완료/회원정보.csv (400000 rows)
📄 저장 중: drive/MyDrive/고급프로그래밍/train/1.회원정보/201807_train_회원정보.parquet → drive/MyDrive/고급프로그래밍/병합완료/회원정보.csv (400000 rows)
📄 저장 중: drive/MyDrive/고급프로그래밍/train/2.신용정보/201812_train_신용정보.parquet → drive/MyDrive/고급프로그래밍/병합완료/신용정보.csv (400000 rows)
📄 저장 중: drive/MyDrive/고급프로그래밍/train/2.신용정보/201811_train_신용정보.parquet → drive/MyDrive/고급프로그래밍/병합완료/신용정보.

## 기본 정보 확인
- 행/열 수, 컬럼 이름, 데이터 타입, 유니크 값 수
- 어떤 정보가 어떤 단위로 정리되어 있는지 구조 파악

In [2]:
# 필요한 라이브러리 불러오기
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# 폴더 경로 및 데이터셋 이름 정의
base_path = "/content/drive/MyDrive/고급프로그래밍/train"
folder_title = ["회원정보", "신용정보", "승인매출정보", "청구입금정보", "잔액정보", "채널정보", "마케팅정보", "성과정보"]
folders = [f"{i}.{name}" for i, name in enumerate(folder_title, start=1)]

# 각 폴더의 parquet 파일 읽어오기
dfs = {}
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    for file in os.listdir(folder_path):
        if file.endswith(".parquet"):
            df_name = folder.split(".")[1]
            file_path = os.path.join(folder_path, file)
            dfs[df_name] = pd.read_parquet(file_path)
            print(f"✅ [{df_name}] 로드 완료: {file}")

✅ [회원정보] 로드 완료: 201812_train_회원정보.parquet
✅ [회원정보] 로드 완료: 201811_train_회원정보.parquet
✅ [회원정보] 로드 완료: 201810_train_회원정보.parquet
✅ [회원정보] 로드 완료: 201809_train_회원정보.parquet
✅ [회원정보] 로드 완료: 201808_train_회원정보.parquet
✅ [회원정보] 로드 완료: 201807_train_회원정보.parquet
✅ [신용정보] 로드 완료: 201812_train_신용정보.parquet
✅ [신용정보] 로드 완료: 201811_train_신용정보.parquet
✅ [신용정보] 로드 완료: 201810_train_신용정보.parquet
✅ [신용정보] 로드 완료: 201809_train_신용정보.parquet
✅ [신용정보] 로드 완료: 201808_train_신용정보.parquet
✅ [신용정보] 로드 완료: 201807_train_신용정보.parquet
✅ [승인매출정보] 로드 완료: 201812_train_승인매출정보.parquet
✅ [승인매출정보] 로드 완료: 201811_train_승인매출정보.parquet
✅ [승인매출정보] 로드 완료: 201810_train_승인매출정보.parquet
✅ [승인매출정보] 로드 완료: 201809_train_승인매출정보.parquet
✅ [승인매출정보] 로드 완료: 201808_train_승인매출정보.parquet
✅ [승인매출정보] 로드 완료: 201807_train_승인매출정보.parquet
✅ [청구입금정보] 로드 완료: 201812_train_청구정보.parquet
✅ [청구입금정보] 로드 완료: 201811_train_ᄎ

- 데이터 프레임 요약

In [8]:
for name, df in dfs.items():
    print("="*80)
    print(f"📁 [{name}] 데이터 프레임 요약")
    print("="*80)

    print("▶ 데이터 shape:", df.shape, "\n")
    print("▶ 컬럼 타입:\n", df.dtypes, "\n")
    print("▶ 결측치 수:\n", df.isnull().sum(), "\n")
    print("▶ 상위 5개 샘플:\n", df.head(), "\n")

📁 [회원정보] 데이터 프레임 요약
▶ 데이터 shape: (400000, 78) 

▶ 컬럼 타입:
 기준년월              int64
ID               object
남녀구분코드            int64
연령               object
Segment          object
                  ...  
우수회원면제카드수_B0M    object
기타면제카드수_B0M      object
카드신청건수            int64
Life_Stage       object
최종카드발급경과월         int64
Length: 78, dtype: object 

▶ 결측치 수:
 기준년월             0
ID               0
남녀구분코드           0
연령               0
Segment          0
                ..
우수회원면제카드수_B0M    0
기타면제카드수_B0M      0
카드신청건수           0
Life_Stage       0
최종카드발급경과월        0
Length: 78, dtype: int64 

▶ 상위 5개 샘플:
      기준년월            ID  남녀구분코드   연령 Segment  회원여부_이용가능  회원여부_이용가능_CA  \
0  201807  TRAIN_000000       2  40대       D          1             1   
1  201807  TRAIN_000001       1  30대       E          1             1   
2  201807  TRAIN_000002       1  30대       C          1             1   
3  201807  TRAIN_000003       2  40대       D          1             1   
4  201807  TRAIN_000004   

- 이상치 및 결측치 탐지

In [9]:
import numpy as np

for name, df in dfs.items():
    print("="*100)
    print(f"📁 [{name}] 데이터 품질 점검")
    print("="*100)

    # ✅ 결측치 탐지
    missing_count = df.isnull().sum()
    missing_cols = missing_count[missing_count > 0]

    if missing_cols.empty:
        print("✅ 결측치 없음")
    else:
        print("⚠️ 결측치 존재:")
        for col, count in missing_cols.items():
            ratio = (count / len(df)) * 100
            print(f" - {col}: {count}개 ({ratio:.2f}%)")

    # ✅ 이상치 탐지 (Z-score 기반)
    num_df = df.select_dtypes(include='number')
    outlier_report = []

    for col in num_df.columns:
        col_data = num_df[col].dropna()
        if col_data.std() == 0:
            continue  # 정적 값 제외

        z_scores = np.abs((col_data - col_data.mean()) / col_data.std())
        outlier_count = (z_scores > 3).sum()
        outlier_ratio = (outlier_count / len(col_data)) * 100

        if outlier_count > 0:
            outlier_report.append((col, outlier_count, outlier_ratio))

    if not outlier_report:
        print("✅ 이상치 없음")
    else:
        print("⚠️ 이상치 존재:")
        for col, count, ratio in sorted(outlier_report, key=lambda x: -x[2]):
            print(f" - {col}: {count}개 ({ratio:.2f}%)")

    print("\n")

📁 [회원정보] 데이터 품질 점검
⚠️ 결측치 존재:
 - 가입통신회사코드: 67364개 (16.84%)
 - 직장시도명: 40807개 (10.20%)
 - _1순위신용체크구분: 2147개 (0.54%)
 - _2순위신용체크구분: 157866개 (39.47%)
 - 최종유효년월_신용_이용가능: 32828개 (8.21%)
 - 최종유효년월_신용_이용: 79318개 (19.83%)
 - 최종카드발급일자: 3007개 (0.75%)
⚠️ 이상치 존재:
 - 동의여부_한도증액안내: 38940개 (9.74%)
 - 탈회횟수_발급1년이내: 25983개 (6.50%)
 - 소지카드수_유효_신용: 19257개 (4.81%)
 - 유효카드수_신용: 17510개 (4.38%)
 - 회원여부_이용가능: 17180개 (4.29%)
 - 소지카드수_이용가능_신용: 16689개 (4.17%)
 - 탈회횟수_발급6개월이내: 15522개 (3.88%)
 - 유효카드수_체크: 15232개 (3.81%)
 - 이용가능카드수_체크: 15094개 (3.77%)
 - _2순위카드이용건수: 11329개 (2.83%)
 - 유효카드수_신용_가족: 11262개 (2.82%)
 - 이용가능카드수_신용_가족: 11196개 (2.80%)
 - _2순위카드이용금액: 10486개 (2.62%)
 - 이용금액_R3M_신용_가족: 10464개 (2.62%)
 - 이용금액_R3M_신용: 10304개 (2.58%)
 - 이용카드수_신용_가족: 10231개 (2.56%)
 - 이용금액_R3M_신용체크: 9716개 (2.43%)
 - _1순위카드이용금액: 8560개 (2.14%)
 - 회원여부_연체: 8313개 (2.08%)
 - 이용금액_R3M_체크: 7639개 (1.91%)
 - 이용카드수_신용: 6713개 (1.68%)
 - 유효카드수_신용체크: 5587개 (1.40%)
 - _1순위카드이용건수: 5317개 (1.33%)
 - 이용가능카드수_신용체크: 5011개 (1.25%)
 - 소지여부_신용: 2999개 (0.75

- treshhold = 70%
- IQR 방식으로 이상치 제거

In [10]:
def remove_outliers_iqr(df, threshold=1.5):
    df_clean = df.copy()
    for col in df_clean.select_dtypes(include='number').columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1

        # 이상치 범위 설정
        lower = Q1 - threshold * IQR
        upper = Q3 + threshold * IQR

        # 이상치 제거
        df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]

    return df_clean


# 정제된 결과를 다시 dfs에 반영
for name, df in dfs.items():
    print("="*100)
    print(f"📁 [{name}] 이상치 및 결측치 제거")
    print("="*100)

    before_shape = df.shape

    # 1. 결측치 70% 이상 있는 행 제거
    thresh = int(df.shape[1] * 0.7)
    df_clean = df.dropna(thresh=thresh)

    # 2. IQR 기반 이상치 제거
    df_clean = remove_outliers_iqr(df_clean)

    after_shape = df_clean.shape
    dfs[name] = df_clean  # 덮어쓰기

    print(f"✅ 정제 완료: {before_shape} → {after_shape}\n")

📁 [회원정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 78) → (30849, 78)

📁 [신용정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 42) → (9781, 42)

📁 [승인매출정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 406) → (0, 406)

📁 [청구입금정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 46) → (47417, 46)

📁 [잔액정보] 이상치 및 결측치 제거
✅ 정제 완료: (400000, 82) → (48, 82)

📁 [채널정보] 이상치 및 결측치 제거


KeyboardInterrupt: 